In [1]:
target_url = "https://bsr.sudrf.ru/bigs/portal.html#%7B%22type%22:%22MULTIQUERY%22,%22multiqueryRequest%22:%7B%22queryRequests%22:%5B%7B%22type%22:%22Q%22,%22request%22:%22%7B%5C%22mode%5C%22:%5C%22EXTENDED%5C%22,%5C%22typeRequests%5C%22:%5B%7B%5C%22fieldRequests%5C%22:%5B%7B%5C%22name%5C%22:%5C%22case_document_result_date%5C%22,%5C%22operator%5C%22:%5C%22B%5C%22,%5C%22query%5C%22:%5C%222018-01-01T00:00:00%5C%22,%5C%22sQuery%5C%22:%5C%222018-12-31T00:00:00%5C%22%7D,%7B%5C%22name%5C%22:%5C%22case_document_category_article_cat%5C%22,%5C%22operator%5C%22:%5C%22SEW%5C%22,%5C%22query%5C%22:%5C%22%D0%A1%D1%82%D0%B0%D1%82%D1%8C%D1%8F%20229.1%20%D0%A7%D0%B0%D1%81%D1%82%D1%8C%203%5C%22%7D%5D,%5C%22mode%5C%22:%5C%22AND%5C%22,%5C%22name%5C%22:%5C%22common%5C%22,%5C%22typesMode%5C%22:%5C%22AND%5C%22%7D%5D%7D%22,%22operator%22:%22AND%22,%22queryRequestRole%22:%22CATEGORIES%22%7D%5D%7D,%22sorts%22:%5B%7B%22field%22:%22score%22,%22order%22:%22desc%22%7D%5D,%22simpleSearchFieldsBundle%22:%22default%22,%22filterGroups%22:%5B%7B%22filterQueries%22:%5B%7B%22field%22:%22case_court_type%22,%22query%22:%22%D0%A0%D0%B0%D0%B9%D0%BE%D0%BD%D0%BD%D1%8B%D0%B9,%20%D0%B3%D0%BE%D1%80%D0%BE%D0%B4%D1%81%D0%BA%D0%BE%D0%B9,%20%D0%BC%D0%B5%D0%B6%D1%80%D0%B0%D0%B9%D0%BE%D0%BD%D0%BD%D1%8B%D0%B9%20%D1%81%D1%83%D0%B4%22,%22not%22:false%7D%5D,%22groupMode%22:%22OR%22,%22not%22:false%7D%5D%7D"

In [66]:
logging?

Object `logging` not found.


In [194]:
import logging
import selenium as se
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from utils import LazyProperty
from functools import partial

In [198]:
EC.presence_of_element_located?

Init signature: EC.presence_of_element_located(locator)
Docstring:     
An expectation for checking that an element is present on the DOM
of a page. This does not necessarily mean that the element is visible.
locator - used to find the element
returns the WebElement once it is located
File:           ~/.virtualenvs/semantic_env_36/lib/python3.6/site-packages/selenium/webdriver/support/expected_conditions.py
Type:           type


In [204]:
%%time
url = target_url
options = se.webdriver.ChromeOptions()
options.add_argument('headless')
driver = se.webdriver.Chrome(chrome_options=options)
driver.get(url)
driver.implicitly_wait(30)
elements = driver.find_elements(by=By.CSS_SELECTOR, value=".resultHeader.openCardLink")
elements

/home/tonko22/.virtualenvs/semantic_env_36/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: use options instead of chrome_options
  after removing the cwd from sys.path.


CPU times: user 20 ms, sys: 0 ns, total: 20 ms
Wall time: 10.2 s


In [298]:
logger = logging.getLogger()

class Scraper:
    patterns = {
        "bsr.sudrf.ru": "Уголовное дело"
    }

    element_css_tags = {
        "case_url" : ".resultHeader.openCardLink",
        "file_url": ".bigField"
    }

    def __init__(self, url, pattern_key: str="bsr.sudrf.ru"):
        self.url = url
        self.options = se.webdriver.ChromeOptions()
        self.options.add_argument('headless')
        self.driver = se.webdriver.Chrome(chrome_options=self.options)
        self.driver.implicitly_wait(30)
        try:
            self.pattern = self.patterns[pattern_key]
        except BaseException as e:
            logger.warning(e)
            
        self.driver.get(self.url)
        
    def get_elements_by_css_selector(self, element_name: str):
        css_tag = self.element_css_tags.get(element_name)
        elements = self.driver.find_elements(by=By.CSS_SELECTOR, value=css_tag)
        return list(elements)

    @LazyProperty
    def case_url_elements(self) -> list:
        return self.get_elements_by_css_selector("case_url")
    
    def get_url(self, element: se.webdriver.remote.webelement.WebElement) -> str:
        """ Parse current web page"""
        return element.get_attribute("href")
    
    @LazyProperty
    def current_page_case_urls(self) -> list:
        return list(map(self.get_url, self.case_url_elements))
    
    @LazyProperty
    def file_url(self):
        return list(map(self.get_url, self.get_elements_by_css_selector("file_url")))
        
    def screenshot(self, element: se.webdriver.remote.webelement.WebElement):
        """ Saves element screenshot to current directory """
        fname = element.text+".png"
        element.screenshot(fname)
    
    def go_to_next_page(self):
        pass


search_page = Scraper(target_url)
case_page_url = search_page.current_page_case_urls[0]
case_page = Scraper(case_page_url)

/home/tonko22/.virtualenvs/semantic_env_36/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: use options instead of chrome_options
/home/tonko22/.virtualenvs/semantic_env_36/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: use options instead of chrome_options


In [ ]:
case_page.case_url_elements

In [293]:
case_page.get_case_url(case_page.case_url_elements[0])

IndexError: list index out of range

In [300]:
case_page.file_url

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=60413): Max retries exceeded with url: /session/33c5c5476502c1d615740f2aece29bda/elements (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f6d6e6e7400>: Failed to establish a new connection: [Errno 111] Connection refused',))

In [ ]:
file_page = Scraper(case_page.file_url)
text_elem = file_page.driver.find_elements(by=By.CSS_SELECTOR, value=".width100.height100.field-iframe")[0]

In [ ]:
text_elem.find_element_by_tag_name("p")

In [284]:
text_span_elem = text_elem.find_element_by_xpath("//span")

In [287]:
# elem = text_elem.find_element_by_partial_link_text("TEXT-INDENT")

In [285]:
text_span_elem.text

'Найти в деле'

In [273]:
text_elem.as_json()

AttributeError: 'list' object has no attribute 'as_json'

In [ ]:
file_page.

In [257]:
case_page.file_url

'https://bsr.sudrf.ru/bigs/showDocument.html?id=0d9bd36643ac3b90550ac542d06543dc&shard=%D0%92%D1%81%D0%B5%20%D0%B4%D0%B5%D0%BB%D0%B0%20(%D0%BD%D0%BE%D0%B2%D0%BE%D0%B5)&fieldName=case_user_document_text_tag&from=p&r=%7B%22start%22:0,%22rows%22:10,%22uid%22:%229dc17a61-f27f-49f1-a56d-0fe1f9fb5fc2%22,%22type%22:%22MULTIQUERY%22,%22multiqueryRequest%22:%7B%22queryRequests%22:%5B%7B%22type%22:%22Q%22,%22request%22:%22%7B%5C%22mode%5C%22:%5C%22EXTENDED%5C%22,%5C%22typeRequests%5C%22:%5B%7B%5C%22fieldRequests%5C%22:%5B%7B%5C%22name%5C%22:%5C%22case_document_result_date%5C%22,%5C%22operator%5C%22:%5C%22B%5C%22,%5C%22query%5C%22:%5C%222018-01-01T00:00:00%5C%22,%5C%22sQuery%5C%22:%5C%222018-12-31T00:00:00%5C%22%7D,%7B%5C%22name%5C%22:%5C%22case_document_category_article_cat%5C%22,%5C%22operator%5C%22:%5C%22SEW%5C%22,%5C%22query%5C%22:%5C%22%D0%A1%D1%82%D0%B0%D1%82%D1%8C%D1%8F%20229.1%20%D0%A7%D0%B0%D1%81%D1%82%D1%8C%203%5C%22%7D%5D,%5C%22mode%5C%22:%5C%22AND%5C%22,%5C%22name%5C%22:%5C%22common%5

In [252]:
case_page.driver.find_element(by=By.CSS_SELECTOR, value=".bigField")

<selenium.webdriver.remote.webelement.WebElement (session="9495bb1344d4c51956905ae94659e0ea", element="0.13760453720084875-1")>

In [234]:
case_page.current_page_case_urls

[]

In [ ]:
case_page = Scraper(search_page.current_page_case_urls[0])

In [224]:
el_0 = case_page.elements

In [227]:
len(el_0)

0

In [ ]:
el_0.

In [215]:
el = list(search_page.elements)[0]

In [219]:
el.text

'Уголовное дело 1-53/2018'

In [ ]:
Scraper

In [79]:
element.screenshot("element.png")

True